# Parallel TED subtitles dataset

Let's construct the TED subtitles dataset.

In [1]:
import re
import numpy as np
import pickle
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import codecs
import os,glob
import glob
import re

### At first let's try to get transriptions from the official site (not working).

In [82]:
def enlist_talk_names(path,dict_):
    with urllib.request.urlopen(path) as url:
        r = url.read()
        soup = BeautifulSoup(r)
        talks= soup.find_all("a",class_='')
        for i in talks:
            if i.attrs['href'].find('/talks/')==0 and dict_.get(i.attrs['href'])!=1:
                dict_[i.attrs['href']]=1

    return dict_

In [76]:
#all_talk_names={}
#for i in range(1,10000):
#    path='https://www.ted.com/talks?page=%d'%(i)
#    all_talk_names=enlist_talk_names(path,all_talk_names)

In [ ]:
def extract_talk(path,talk_name):
    r=urllib.urlopen(path).read()
    soup=BeautifulSoup(r)
    df=pd.DataFrame()
    print(path)
    for i in soup.findAll('link'):
        if i.get('href')!=None and i.attrs['href'].find('?language=')!=-1:
            lang=i.attrs['hreflang']
            path=i.attrs['href']
            r1=urllib.urlopen(path).read()
            soup1=BeautifulSoup(r1)
            time_frame=[]
            text_talk=[]
            for i in soup1.findAll('span',class_='talk-transcript__fragment'):
                time_frame.append(i.attrs['data-time'])
                text_talk.append(i.text.replace('\n',' '))
            #print len(time_frame),len(text_talk)
            df1=pd.DataFrame()
            df1[lang]=text_talk
            df1[lang+'_time_frame']=time_frame
            df=pd.concat([df,df1],axis=1)
    df.to_csv(talk_name+'.csv',sep='\t',encoding='utf-8')

In [80]:
for i in all_talk_names:
    extract_talk('https://www.ted.com'+i+'/transcript',i[7:])

TED is blocking multiple connections. Let's find another way.

### Search parallel corpuses on Github (not working)

Corpus at this link seems to be great. 

https://github.com/ajinkyakulkarni14/TED-Multilingual-Parallel-Corpus/tree/master/Bilingual_Parallel_Corpus

In [50]:
corpus1 = pd.read_csv('D:\ml\zh-ru\parallel_corpus_1.txt', delimiter=':', names=['sent_id','lang_code','text'])
corpus2 = pd.read_csv('D:\ml\zh-ru\parallel_corpus_2.txt', delimiter=':', names=['sent_id','lang_code','text'])

In [56]:
corpus1[corpus1.sent_id == "132112"]

,sent_id,lang_code,text
1376490,132112,ar,"و هم الآن يصنعون خليطا,"
1376491,132112,nl,"Ze zijn nu bezig een cocktail,"
1376492,132112,fr,"Ils développent actuellement un cocktail,"
1376493,132112,de,"Sie machen gerade einen Cocktail,"
1376494,132112,he,"כיום הם מייצרים קוקטייל,"
1376495,132112,it,In questo momento si stanno preparando
1376496,132112,ru,Они сейчас готовят коктейль
1376497,132112,es,"Actualmente están haciendo un cóctel,"
1376498,132112,ar,"و هم الآن يصنعون خليطا,"
1376499,132112,pt-br,"Atualmente eles estão fazendo um coquetel,"


In [57]:
corpus2[corpus2.sent_id == "132112"]

,sent_id,lang_code,text
555789,132112,zh-cn,但是有一个声音一直在呼唤我：“来，别离开我。“
555790,132112,zh-tw,但有個聲音不斷喚著我：「拜託，堅持下去。」
555791,132112,ja,でも別の声が言います
555793,132112,ko,하지만 또다른 목소리가


In [53]:
corpus1.head()

,sent_id,lang_code,text
0,1,ar,بالنسبة لي، تبدأ هذه القصة قبل حوالي 15 عاماً ...
1,1,nl,Voor mij begint dit verhaal ongeveer 15 jaar g...
2,1,fr,"Pour moi, cette histoire a commencé il y a 15 ..."
3,1,de,Für mich beginnt diese Geschichte vor ungefähr...
4,1,he,"עבורי, הסיפור הזה התחיל בערך לפני 15 שנים,"


In [18]:
corpus2.head()

,sent_id,lang_code,text
0,1,zh-cn,对于我来说，这个故事是15年前开始的。
1,1,zh-tw,大約15年前
2,1,ja,話は15年前にさかのぼります
3,1,ko,"제게 있어서, 이 이야기는 15년 전에 시작했죠,"
4,2,zh-cn,当时我是芝加哥大学安养院的医生，


In [59]:
corpus1[corpus1.lang_code == 'ru'].shape

(345542, 3)

In [24]:
corpus2[corpus2.lang_code == 'zh-cn'].shape

(384109, 3)

In [39]:
zhru_df = pd.merge(corpus1[corpus1.lang_code == 'ru'].loc[:,['sent_id', 'text']], \
                   corpus2[corpus2.lang_code == 'zh-cn'].loc[:,['sent_id', 'text']], on='sent_id')

zhru_df.columns = ['sent_id', 'ru','zh']

In [60]:
zhru_df.head()

,sent_id,ru,zh
0,1,"Для меня всё началось лет 15 тому назад, когда...",对于我来说，这个故事是15年前开始的。
1,2,в хосписе при Чикагском Университете.,当时我是芝加哥大学安养院的医生，
2,3,Я оказывал помощь умирающим и их семьям,在芝加哥的南边地区
3,4,в районе Саут Сайд города Чикаго.,照顾临终的病人和他们的亲属。
4,5,"И я видел, что происходит на конечной стадии з...",我借此来观察疾病晚期


Unfortunately, half of the files are on the Git LFS storage and not available because author didn't upgrade the plan. And from existed files we can make parallel corpus, but it is not reliable (sentence ID's are not corresponding in a right way).

### Kaggle datasets

So, I've found the following dataset on Kaggle.

https://www.kaggle.com/goweiting/ted-talks-transcript

It's a large bunch of the subtitle files that we need to process to make the useful stuff.

In [72]:
path = "D:\\ml\\ted-talks-transcript\\tedDirector\\subtitles"

In [73]:
id = 0
subtitles = {}

for d in [dirpath for (dirpath, dirnames, filenames) in os.walk(path)][1:]:
    types = ['\*.ru.vtt', '\*.zh-CN.vtt']
    pair = []
    
    for t in types:
        files = glob.glob(d + t)
        if len(files) == 1:
            pair.extend(files)
    
    if len(pair) == 2:
        subtitles[id] = pair
        id += 1

In [74]:
len(subtitles)

2309

In [75]:
subtitles[0]

['D:\\ml\\ted-talks-transcript\\tedDirector\\subtitles\\-0v7mTvJ8M4\\-0v7mTvJ8M4.ru.vtt',
 'D:\\ml\\ted-talks-transcript\\tedDirector\\subtitles\\-0v7mTvJ8M4\\-0v7mTvJ8M4.zh-CN.vtt']

In [92]:
output_ru = 'D:\\ml\\ted-talks-transcript\\output_ru.txt'
output_zh = 'D:\\ml\\ted-talks-transcript\\output_zh.txt'
tokens_ru = 'D:\\ml\\ted-talks-transcript\\tokens_ru.txt'
tokens_zh = 'D:\\ml\\ted-talks-transcript\\tokens_zh.txt'

with open(output_ru, mode='w', encoding='utf-8') as out_ru, open(output_zh, mode='w', encoding='utf-8') as out_zh, \
        open(tokens_ru, mode='w', encoding='utf-8') as tok_ru, open(tokens_zh, mode='w', encoding='utf-8') as tok_zh:
    for id in [id for id in subtitles]:       
        strings = {}
        textlines = {'ru': [], 'zh':[]}

        for file in subtitles[id]:
            print(file)
            timestamp = ''
            with open(file, mode='r', encoding='utf-8') as f:
                isRu = bool(re.search('ru\.vtt$', file))
                if isRu:
                    lang = 'ru'
                else:
                    lang = 'zh'

                for line in f.readlines():
                    isTimestamp = re.match("^\d\d:\d\d:\d\d\.\d\d\d.*", line)

                    if isTimestamp:

                        if timestamp in strings:
                            strings[timestamp][lang] = ' '.join(textlines[lang])
                            textlines = {'ru': [], 'zh':[]}
                            #print('timestamp')

                        #it comes out that the second date in timing is not reliable. taking the start time as key
                        timestamp = line[:16]
                        if timestamp not in strings:
                            strings[timestamp] = {'ru': '', 'zh':''}
                    else: 
                        if timestamp in strings:
                            #print('true', lang)
                            if len(line.strip()) > 0:
                                textlines[lang].append(line.replace('\t',' ').strip())
                            #print(textlines[lang])

        for ts in strings:
            if (len(strings[ts]['ru']) > 2) & (len(strings[ts]['zh']) > 2):
                str_ru = strings[ts]['ru']
                str_zh = strings[ts]['zh']
                
                out_ru.write(str_ru + '\n')
                out_zh.write(str_zh + '\n')
                
                #print(' '.join(word_tokenize(str_ru)))
                #print(' '.join(jieba.lcut(str_zh)))
                
                tok_ru.write(' '.join(word_tokenize(str_ru)) + '\n')
                tok_zh.write(' '.join(jieba.lcut(str_zh)) + '\n')
                

D:\ml\ted-talks-transcript\tedDirector\subtitles\-0v7mTvJ8M4\-0v7mTvJ8M4.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\-0v7mTvJ8M4\-0v7mTvJ8M4.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\-2Dj9M71JAc\-2Dj9M71JAc.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\-2Dj9M71JAc\-2Dj9M71JAc.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\-3casC77XSc\-3casC77XSc.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\-3casC77XSc\-3casC77XSc.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\-aouBn7IKIo\-aouBn7IKIo.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\-aouBn7IKIo\-aouBn7IKIo.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\-bdf1NnDZ8M\-bdf1NnDZ8M.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\-bdf1NnDZ8M\-bdf1NnDZ8M.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\-Bn6Gel7yEs\-Bn6Gel7yEs.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\-Bn6Gel7yEs\-Bn6Gel7yEs.zh-CN.vtt
D:\ml\ted-talks-transc

D:\ml\ted-talks-transcript\tedDirector\subtitles\0YNeyBANrTI\0YNeyBANrTI.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\0YNeyBANrTI\0YNeyBANrTI.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\0ZhL7P7w3as\0ZhL7P7w3as.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\0ZhL7P7w3as\0ZhL7P7w3as.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\0zvrGiPkVcs\0zvrGiPkVcs.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\0zvrGiPkVcs\0zvrGiPkVcs.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\0_UTgoPUTLQ\0_UTgoPUTLQ.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\0_UTgoPUTLQ\0_UTgoPUTLQ.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\10MvdLL2EpY\10MvdLL2EpY.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\10MvdLL2EpY\10MvdLL2EpY.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\12OAr0lt4bk\12OAr0lt4bk.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\12OAr0lt4bk\12OAr0lt4bk.zh-CN.vtt
D:\ml\ted-talks-transc

D:\ml\ted-talks-transcript\tedDirector\subtitles\2HgL5OFip-0\2HgL5OFip-0.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\2HgL5OFip-0\2HgL5OFip-0.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\2K-P7Hv_Zwg\2K-P7Hv_Zwg.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\2K-P7Hv_Zwg\2K-P7Hv_Zwg.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\2L4B-Vpvx1A\2L4B-Vpvx1A.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\2L4B-Vpvx1A\2L4B-Vpvx1A.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\2lJvr5UL2pQ\2lJvr5UL2pQ.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\2lJvr5UL2pQ\2lJvr5UL2pQ.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\2m9kC1yRnLQ\2m9kC1yRnLQ.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\2m9kC1yRnLQ\2m9kC1yRnLQ.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\2MYsx6WArKY\2MYsx6WArKY.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\2MYsx6WArKY\2MYsx6WArKY.zh-CN.vtt
D:\ml\ted-talks-transc

D:\ml\ted-talks-transcript\tedDirector\subtitles\3qRtDnsnSwk\3qRtDnsnSwk.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\3qRtDnsnSwk\3qRtDnsnSwk.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\3QZp6smeSQA\3QZp6smeSQA.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\3QZp6smeSQA\3QZp6smeSQA.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\3r1IPsldbBg\3r1IPsldbBg.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\3r1IPsldbBg\3r1IPsldbBg.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\3u_QlcGCHUU\3u_QlcGCHUU.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\3u_QlcGCHUU\3u_QlcGCHUU.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\3vfuCPFb8wk\3vfuCPFb8wk.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\3vfuCPFb8wk\3vfuCPFb8wk.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\3wxBTEo8-T8\3wxBTEo8-T8.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\3wxBTEo8-T8\3wxBTEo8-T8.zh-CN.vtt
D:\ml\ted-talks-transc

D:\ml\ted-talks-transcript\tedDirector\subtitles\5ODzO7Lz_pw\5ODzO7Lz_pw.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\5ODzO7Lz_pw\5ODzO7Lz_pw.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\5P04stEjJ9E\5P04stEjJ9E.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\5P04stEjJ9E\5P04stEjJ9E.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\5qaktKVL_zM\5qaktKVL_zM.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\5qaktKVL_zM\5qaktKVL_zM.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\5riPKdpuoiI\5riPKdpuoiI.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\5riPKdpuoiI\5riPKdpuoiI.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\5tal0NkWW1E\5tal0NkWW1E.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\5tal0NkWW1E\5tal0NkWW1E.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\5tUn_uoYKoE\5tUn_uoYKoE.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\5tUn_uoYKoE\5tUn_uoYKoE.zh-CN.vtt
D:\ml\ted-talks-transc

...

D:\ml\ted-talks-transcript\tedDirector\subtitles\xuAAPsiD768\xuAAPsiD768.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\xuAAPsiD768\xuAAPsiD768.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\xukDIWFMU9Y\xukDIWFMU9Y.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\xukDIWFMU9Y\xukDIWFMU9Y.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\Xv2gHiX9Bqw\Xv2gHiX9Bqw.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\Xv2gHiX9Bqw\Xv2gHiX9Bqw.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\XVXmYD0UPRQ\XVXmYD0UPRQ.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\XVXmYD0UPRQ\XVXmYD0UPRQ.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\xxXa9pxwzoY\xxXa9pxwzoY.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\xxXa9pxwzoY\xxXa9pxwzoY.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\XyCY6mjWOPc\XyCY6mjWOPc.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\XyCY6mjWOPc\XyCY6mjWOPc.zh-CN.vtt
D:\ml\ted-talks-transc

D:\ml\ted-talks-transcript\tedDirector\subtitles\yhHePLaMP_k\yhHePLaMP_k.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\yhMi0tkYyT8\yhMi0tkYyT8.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\yhMi0tkYyT8\yhMi0tkYyT8.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\YhvfOlPYifY\YhvfOlPYifY.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\YhvfOlPYifY\YhvfOlPYifY.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\yhyzTTaMN7E\yhyzTTaMN7E.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\yhyzTTaMN7E\yhyzTTaMN7E.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\yioZhHe1i5M\yioZhHe1i5M.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\yioZhHe1i5M\yioZhHe1i5M.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\YiXd_9DFCOQ\YiXd_9DFCOQ.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\YiXd_9DFCOQ\YiXd_9DFCOQ.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\YKACzIrog24\YKACzIrog24.ru.vtt
D:\ml\ted-talks-transc

D:\ml\ted-talks-transcript\tedDirector\subtitles\z5L_vPkorBU\z5L_vPkorBU.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\Z5TUpNZqjM8\Z5TUpNZqjM8.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\Z5TUpNZqjM8\Z5TUpNZqjM8.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\Z7CtKDNf2RI\Z7CtKDNf2RI.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\Z7CtKDNf2RI\Z7CtKDNf2RI.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\z9sa2ZAqHl0\z9sa2ZAqHl0.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\z9sa2ZAqHl0\z9sa2ZAqHl0.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\zAFcV7zuUDA\zAFcV7zuUDA.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\zAFcV7zuUDA\zAFcV7zuUDA.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\zAvNlh2Z0GI\zAvNlh2Z0GI.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\zAvNlh2Z0GI\zAvNlh2Z0GI.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\ZB50BfYlsDc\ZB50BfYlsDc.ru.vtt
D:\ml\ted-talks-transc

D:\ml\ted-talks-transcript\tedDirector\subtitles\zuYzOn0U2PY\zuYzOn0U2PY.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\zuYzOn0U2PY\zuYzOn0U2PY.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\zvwQXwJyaj0\zvwQXwJyaj0.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\zvwQXwJyaj0\zvwQXwJyaj0.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\Zw7zA-RV5yE\Zw7zA-RV5yE.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\Zw7zA-RV5yE\Zw7zA-RV5yE.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\zw9NXY3xOOI\zw9NXY3xOOI.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\zw9NXY3xOOI\zw9NXY3xOOI.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\zwjLQgNd7XM\zwjLQgNd7XM.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\zwjLQgNd7XM\zwjLQgNd7XM.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\zwpiI18TBdE\zwpiI18TBdE.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\zwpiI18TBdE\zwpiI18TBdE.zh-CN.vtt
D:\ml\ted-talks-transc

D:\ml\ted-talks-transcript\tedDirector\subtitles\_xMWqUe8FdU\_xMWqUe8FdU.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\_ZW-8-NCKMw\_ZW-8-NCKMw.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\_ZW-8-NCKMw\_ZW-8-NCKMw.zh-CN.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\__zRf_JTNW4\__zRf_JTNW4.ru.vtt
D:\ml\ted-talks-transcript\tedDirector\subtitles\__zRf_JTNW4\__zRf_JTNW4.zh-CN.vtt


In [86]:
#707839
#552370

#### Refinining

In [3]:
#df_corpus = pd.read_csv('D:\\ml\\ted-talks-transcript\\output.txt', delimiter='\t', names=['ru', 'zh'])

In [79]:
ru_series = pd.read_csv(output_ru, names=['ru'])
zh_series = pd.read_csv(output_zh, names=['zh'])

In [80]:
zh_series.shape

(511839, 1)

In [81]:
ru_series.shape

(539815, 1)

In [18]:
from nltk import word_tokenize
import jieba

In [15]:
word_tokenize('он пошел во двор, чтобы ффф: "фывфыв"!')

['он',
 'пошел',
 'во',
 'двор',
 ',',
 'чтобы',
 'ффф',
 ':',
 '``',
 'фывфыв',
 "''",
 '!']

In [21]:
jieba.lcut('翻译人员: Angelia King 校对人员: Jenny Yang')

['翻译人员',
 ':',
 ' ',
 'Angelia',
 ' ',
 'King',
 ' ',
 '校对',
 '人员',
 ':',
 ' ',
 'Jenny',
 ' ',
 'Yang']

In [64]:
jieba.lcut('表明我在演奏的是哪种声音')

['宗教', '人士', '会', '说', ' ', '"', '胡扯', '"']

In [36]:
df_corpus.ru = df_corpus.ru.apply(lambda x: ' '.join(word_tokenize(x)))

In [65]:
zh_tokenized = pd.Series(name='zh_tokenized')

for x in df_corpus.zh:
    print(x)
    zh_series.add(' '.join(jieba.lcut(x)))
    
df_corpus.join()

翻译人员: Angelia King 校对人员: Jenny Yang
（音乐）
文本：节拍爵士
节拍爵士是
1. 现场循环
2.爵士即兴表演
3.“手势表达”的声效设计
感应器
在每只手上的感应器
读懂手语姿势
亮灯的颜色
表明我在演奏的是哪种声音
红色=鼓声
蓝色=贝司低音
绿色=和弦
橙色=主音
紫色=键盘声
吹口器
一个按钮，
两个吉他拨片
和许多热的粘接剂组成
前镜显示器
是一个智能手机，它显示系统参数。
为什么这样做？
为了震动音乐文化
以至于所有过去的，现在的和未来的音乐类型
可以被研究，和抽象化，得以存在下去。
“节拍爵士音乐家”
和DJ一样变得普通常见
但主要是...
改变未来
而非等待未来。
翻译人员: Yu Liu 校对人员: Rale Don
美国的孩子们长大成人的道路上，
有两个机构在这段旅程上至关重要。
第一个是大家经常听到的：大学。
某些人可能还记得
当你第一次进入大学时的兴奋的感觉
你们中的某些可能现在就在大学
并且正在享受那份兴奋
大学有很多弊端
学费昂贵，所以年轻人负债累累
但总而言之，这是一条康庄大道
年轻人从校园毕业，带着自豪与友情
和许多关于这个世界的知识
或许更重要的是
上大学使得他们能有更好的就业机会
今天我想讨论的是第二个机构
在美国，贯穿了从童年到成年的整个人生经历
那个机构便是监狱
在这段旅程上,相伴着年轻人的
是感化官而不是教师
去法庭受审而不是去教室上课
他们的大三留学之旅是去州立管教所
当他们20多岁时
没有商科的或英语的学位
有的只是犯罪记录
这个机构同样花费甚多
在新泽西，送一个年轻人到监狱的花费
但是这是纳税人买的单
而孩子们得到的只是一个冰冷的牢房单间
和一个永久的印记，阻碍着他们回归家庭
或者寻找工作
越来越多的孩子在这条路上长大成人
尤其在美国，这是因为在过去的四十年里
我们服刑率已经增长了700%
我制作了一张幻灯片
看这儿
这是我们的服刑率
每十万人就有716人服刑
这是其他OECD（经合组织）成员国家的情况
更为重要的是，被送入监狱的孩子往往 家境贫寒
他们大多来自非裔美国人和拉丁裔社区
以至于监狱成为了想要成功的年轻人
实现美国梦的障碍
问题是事实更为糟糕
因为我们不只是把贫困的孩子送入监狱
我们还给他们加上了许多沉重的枷锁，比如诉讼费的负担
比如感化和假释的限制
比如轻微的犯罪通缉
我们

当到你该吃药的时候，它就会自动地膨胀开来。
这样孩子就会去打开药瓶。
而另外这个药是可以让你在身上作画的。
皮内的传输让你很快乐地参与到
这种特别的传输过程中来。
相似地，和Marti Guixe一样，
有很多设计品试图让你参与其中，并且让所有的东西都从你嘴里进入。
因此你可以通过口服尝到犯错的滋味。
我要做的下一个展示是
关于设计和科学间的关系的，
我已多次向你们提及这件事。
我试图寻找的并非隐喻而是共识类的
普遍性的：当前的共同问题，当务之急。
而我认为这一点能使我们在设计的理念上走的更远。
使得设计成为一种启迪，一种方向而不是一种关于形式的指示。
同时我也希望你们都能对此做出响应。
我已经向你们其中一些人发送了邮件。
我们可以从不同的层面上去以形象化的方式展现二者的魅力。
这需要从很小的工作做起，
只有这样才能做出伟大和有意义的事情。
翻译人员: Eva Sun 校对人员: Ray Yang
我要带大家去的第一个地方
是一个天然的深渊，许多人相信它将会被证明为世界最深之渊。
我说相信是因为这个探索过程还未结束。
明年有几个很大的探险活动正在策划中
关于这个我会做些介绍
现在,变化之一是
自从儒勒•凡尔纳
在科幻小说里描绘了地底世界的样子后,
在过去的150年里，科技已使我们能够到达一些地方
这些地方是我们先前完全不知且不能想象的。
而我们现在能相对安全得深入地球几千米。
沿途我们发现了奇异的深渊和岩洞。
那些岩洞大得异乎寻常，
一眼望去能看到几百米。
当我们到了这样的岩洞时，我们经常会在那
待二至四个月，
我们的队伍规模小时二三十人，大时有一百五十人。
许多人问我
你们选什么样的人参加这样的项目？
我们的筛选过程
不是像NASA(美国航空航天局)那么严格，但它又是周详的。
我们寻找的是有能力胜任此工作，能严格约束自己，耐力长久而且体力充沛的人。
如果你想知道的话，这是我们的体能测试。
（笑）
但我们也重视团队精神
和善于解决人际矛盾的能力。
这些能力， 在人们身处僻壤
压力巨大到远远超过人类耐力极限的时候尤其重要。
从洞口看，这绝不像个商业旅游的山洞。
你看到的是二号营地，它搭在J2,不是K2(世界第二高峰）,而是J2.
在那里我们大约距出口两天路程。
它就像一场反方向的高海拔登山之旅，
不同的是你用绳子往下走
这是让人们在这种地下环境中稍微觉得舒服些
要不然

我的重点是说，你们也同样可以对
这些孩子产生重大影响
事实上，说到这里我想起几年前
我去帕洛阿尔托（加州城市）一所学校演讲
来听演讲的是一群11岁左右
的孩子们
我准备对他们做一小时的演讲
这群11岁的孩子们坐在一起围成一个半圆
睁大眼睛看着我，然后我就开始演讲
我身后以一张白板，我在上面
写下一个1和22个0，我说
“看吧，这就是我们能在宇宙里看见
的星星数量，这个数字太大了
我们都不知道怎样表达它。”
然后其中一个孩子举起手来，说
“事实上他是有名字的，
他是十的6次方加4次方加16次方。”对吗？
虽然这个孩子多说了4个数量级
但是毫无疑问，这些孩子们非常聪明
所以我停止了我的演讲
因为他们真正想做的是提问
事实上，我最后对这些孩子们有些评价，我说
“你知道吗，你们这些孩子甚至
比和我一起工作的人们还要聪明。”但是 — （笑声）
他们根本不在乎这个评价
他们只想要知道我的邮箱地址
来问我更多问题。（笑声）
我们只能说，我的工作算是一种特权
因为我们生活在一个特殊的时代
以前的人们根本不能做这种实验
但是在我们的下一代中
我想我们会成功的
所以对我来说，我的工作是个特权，而当我照镜子的时候
我根本看不到我自己
我只看得到我身后的这一代人
这是一些来自于Huff小学4年级的学生
大概两周之前我在那里演讲
我认为如果你能喜爱科学
以及它的运作方式，那么它带来的回报
是无法简单估量的。谢谢大家
翻译人员: Xiao Xiao 校对人员: Louise LIANG
我研究的是
犯罪和恐怖主义的未来。
坦白地说，我感到害怕。
我对我在研究中了解到的事情感到害怕。
我真诚地希望自己能够相信
科技可以带给我们
对我们的承诺中的科技乌托邦
但是，你们看，
我曾经度过了一段与执法有关的职业生涯，
而这让我对事物的看法有了新的认识。
我曾经做过巡街警察，
卧底侦探，
反恐战略师，
也曾经在全球超过70个国家
工作过。
我不得不经历了比我应该遇到的更多的
暴力和社会阴暗面，
而正是这些影响了我的观点。
我的工作会与罪犯和恐怖分子打交道
这实际上是非常有教育意义的。
他们教会了我很多，而我希望可以
与你们分享一些我所观察到的东西。
今天我将展现给你们
那些让我们感到惊奇的
我们所爱的科技的另一面
如果科技由TDE社区掌控
那么它无疑是可以给我们的世界带来巨大改变的
不可思议的工具，
但是

我无法找到问题的答案
然后我明白了
事实上我们对这两个领域的了解都还不够
都还不足以完全弄清真相
化学治疗和心理治疗
都发挥着重要的作用
我也发现抑郁是这样一个东西
深深的嵌入在我们的体内
我们无法将它彻底剥离
它已经嵌入到我们的性格和个性中了
我想说现在我们所用的
治疗抑郁症的方法太可怕了
这些方法没有什么效果
还非常昂贵
并且伴随着无数的副作用
它们简直就是灾难
但我很感激我活在当下
而不是50年前
那个时候还不存在
有效的方法
我希望50年后
人们听到我接受的治疗方法
会震惊于竟然有人愿意忍受
如此原始简单的科学
抑郁是爱的附属品
如果你跟一个人结婚了，然后想
“好吧，如果我的妻子去世了，我会找一个新的，”
那么据我们所知这不叫爱
没有这样一种爱情
可以只感受幸福而不体验失去
这种绝望的幽灵
会成为亲密关系的动力
有三种东西是人们容易混淆的
抑郁，悲伤，难过
悲伤是一种明确的反应
如果你遭遇了不幸并感到极度不快乐
紧接着六个月以后
你还是非常难过，但是生活大致正常了
这很有可能是悲伤
而且它很有可能在最终
一定程度地自我恢复
如果你经历了一次灾难性的打击
然后感觉非常糟糕
并且六个月之后你依然无法正常生活
那么很有可能就是你的抑郁
被这种灾难性的情形触发了
这中变化的过程告诉我们很多信息
人们往往认为抑郁只是难过而已
只是太多太多的难过
太多的悲伤
起因却微不足道
当我开始着手了解抑郁
并且采访那些有过这样经历的人时
我发现有些人
从表面上看来
好像是比较轻微的抑郁
却已经因此彻底丧失行为能力了
另一些人
从他们的描述中得知他们
经历了非常严重的抑郁
他们却能够在抑郁发作的间隙
过着不错的生活
于是我开始研究
到底是什么
使一些人比另一些人能更好地适应
是什么样的机制
让这些人能够幸免？
于是我去探访了一个又一个
经历过抑郁的人
我第一批采访的人中有一个人
把抑郁描述为
一种缓慢的死亡方式
这种说法最初在我听来是好的
因为这告诉我
缓慢的死亡方式
是会以真正的死亡结束的
这不是说着玩的
这是世界上导致机能障碍的主要原因之一
每天都有人因此死去
在我试图了解这些的时候
其中一个我采访的人
是我的挚友
我们已经相识很多年了
她曾经在她大学入学的那一年
有过精神病发作
之后陷入了可怕的抑郁
她患有双相情感障碍
当时叫做躁郁症
她经过多年的化学治疗
病情控制得很好
于

在给你们一个伊朗的例子。
我们都听说过最近那里发生的
“推特革命”(Twitter Revolution)。
但如果你仔细看看，你就会发现
很多网络、博客、推特（Twitter）、Facebook
实际上都是正常运行。
可能访问的速度略有减缓，
但积极分子仍旧可以访问它们，
甚至有人认为不封闭这些网站
事实上是对独裁政府有利的。
原因就在于这些网站们
给了政府开源性的情报信息。
在过去，要想搞清楚伊朗的积极分子们是如何彼此关联的
往往需要花上几个星期，甚至是几个月。
现在你这要登陆他们的Facebook网页
看看谁和谁是好友就可以了。
过去苏联安全局(KGB)以及其他组织
要利用酷刑折磨才能得到这些资料。
现在这些资料在网上随手可得。
（笑声）
但我觉得那些“网托邦”人士
所犯的最大的概念性错误
是关于当今的数码一代，那些在网络上成长起来的人们。
我们常常听说网络积极主义，
听说人们是如何因为互联网而变得更加积极的。
但却很少听说比如“网络享乐主义”之类的说法，
听说人们是如何因为互联网而变得被动消极的。
为什么呢？因为这些“网托邦“人士一味地认为
互联网最终将成为社会变化的催化剂，
将鼓舞青年群众走向街头，
但事实上它却成为了老百姓的新鸦片，
鼓舞人们躲在自己房间里，下载色情片。
但很多人却对这种想法不屑一顾。
每个德黑兰街头造反的数码变节者的背后
都隐藏着
两个只在魔兽世界里反抗的数码俘虏。
这就是现实。不过这并没有什么不好的，
因为互联网的确是给了很多年轻人力量。
它对于他们所产生的社会作用是完全不同的。
如果你看看一些关于年轻人
是如何从互联网中受益的问卷调查，
你就会发现在中国
性生活因为互联网而得以扩展的青少年的数量
比美国多三倍。
所以互联网的确扮演着社会角色，
只不过它不一定会带来任何实质性的政治讨论。
所以我情愿把它看作是一种
网络上的需要层次论——
完全是照抄亚伯拉罕-马斯洛(Abraham Maslow)(需要层次论的创立者)。
但重要的一点是
当你把网络带给那些俄罗斯的边远乡村时，
真正能启发人们上网的
并不会是“人权观察“的报告，
而是色情片，《欲望城市》，
或者是根猫有关的搞笑视频。
你们需要认识到这一点。
对此，我们应该做些什么呢？
我们需要摒弃
人均iPod值的概念，
而是开始思考各种方法
将力量带给知识分子、
不同意见者、非政府机构

...

这个数字每年都在增加直到2010年。
这意味着你会有源源不断的新的年轻人
找上门来说，“请给我们工作，”
和“我不需要大幅度地加工资，只要给我一份工作就行。”
现在呢，这种情况彻底不同了。
这个10年，在中国15到24岁之间的 年轻人比例会下降20％到30％。
这个10年，在中国15到24岁之间的 年轻人比例会下降20％到30％。
那么你会在哪里设立自己的新工厂呢？
你会想到南亚，确实人们在这么做。
人们在考虑巴基斯坦和孟加拉国，
同时也在考虑非洲。
而考虑非洲的原因是
那条黄线正在告诉我们
非洲的年轻人数量
会持续的增加，
一个十年接着一个十年一直到2050年。
而有这么多年轻人加入到任何市场 都会产生一个问题，
而有这么多年轻人加入到任何市场 都会产生一个问题，
特别是当他们是年轻男性的时候。
有时候是有些危险的。
我认为一个非常关键的问题是
那部分人口的教育程度如何？
如果大家看看这条红线，
你会看到在1975年，
在撒哈拉以南的非洲 只有9％的孩子接受了中学教育。
在撒哈拉以南的非洲 只有9％的孩子接受了中学教育。
在撒哈拉以南的非洲 只有9％的孩子接受了中学教育。
你会在70年代中期在撒哈拉以南开设工厂吗？
你会在70年代中期在撒哈拉以南开设工厂吗？
没有人这么做。
他们选择了土耳其和墨西哥
开设纺织工厂，
因为他们的教育水平是
25％到30％之间。
今天，撒哈拉以南的教育水平已经达到
土耳其和墨西哥在1975年的水平。
他们会得到纺织工作
这会让人们远离贫困，
走上通往工业化和小康之路。
那么非洲现在是什么样子呢？
我是这么看非洲的。
看上去有点奇怪，因为我是一个经济学家。
每个小盒子大概是10亿美金，
大家可以看到我对中间的
尼日利亚非常关注。
南非起着很大的作用。
但当我想到未来的时候，
实际上我最感兴趣的是
中部、西部和南部非洲。
如果按人口来看非洲，
东非显得非常突出，
潜力巨大。
我用这些地图给大家展示些其他的。
这里显示着民主和专制制度的对比图。
脆弱的民主是米色。
强大的民主是橙色。
大家可以看到的是大部分的非洲人
都生活在民主制度中。
这有什么关系么？
因为人民想要的是
政治家们争取实现的，
他们并不总是成功，但是他们会努力。
因此你就会有一个持续的良性循环。
在2012年12月的加纳大选上，
两个候选人之间的较量是
关于教育。
一个人提

他们的手被口罩外侧污染了，
然后就去用手擦鼻子。中招了！他们得了SARS。
并不是由于飞沫传播。
如果你现在上网，你能找到那么多的虚假信息。
最后你就会买这种所谓的N95口罩。可笑。
实际上我们都没有一个
防护装备标准给第一批疫情响应者参考，
而他们将是抗击疫情的一线人员。
接下来是达菲。你们很可能已经听说过这种药品，
由Hoffmann-La Roche研制的专利药品。
有迹象表明，
在流行病爆发时它可以给你一些喘息之机。
你应该长期服用这种达菲吗？
其实，达菲的其中一个副作用是引发自杀念头。
一个公共卫生调查分析认为
大规模使用达菲
实际上将对
公共卫生措施起反作用，使事态恶化。
另一个有意思的事情是：
当一个人服用达菲后，只有其中20%
通过新陈代谢适当转化
为人体的活性化合物。
其余的转变为稳定化合物，
在过滤中幸存下来并进入到水系中，
从而使他们暴露给那些能够携带流感病毒的水鸟
给它们以
产生耐药株的可能。
现在我们已经在
越南人际传播
和埃及人际传播的疫情中发现了达菲耐药株。
所以我个人认为将达菲视为
一种特效药
是非常有限的——的确非常有限。
尽管如此，大部分政府
还是将它们的全部流感对策
建立在储备达菲上。
俄罗斯实际上已经储备了足够
95%的俄国人使用的达菲。
我们已经储备了足够30%的人使用的达菲。
我说够用，是指够两个星期用。
之后就只能听天由命了，因为
疫情将持续18-24个月。
一些比较穷的
饱受禽流感冲击的国家建立起储备后，
达菲也已经失效了。它们已经过期了。
从1918年
上一次大规模的疫情中我们能知道什么呢？
联邦政府放弃了大部分职责。
结果留下了一个支离破碎的混乱的管理局面
在整个美国。
每个城市，每个郡，每个州都在各行其是。
和想法大相径庭。
在有些案例中所有的学校，教堂，
和公共集会场所都被关闭了。
18个月的时间里疫情反复了三次，
即使当时并没有商业航空旅行。
第二波变异株极具杀伤力。
第一波时我们还有足够的医护人员。
但是第二波袭来时，
医护人员损失惨重，
我们失去了抗击疫情前线的大部分的医生和护士。
死亡人数总计70万人。
对孕妇来说，这种病毒是100%致命的。
而且我们不是很清楚原因。
大多数的死亡案例介于15-40岁之间——
健康的青壮年。
就像瘟疫一样。
实际死亡人数我们不是很清楚。
保守估计是三千五百万。
这个数字

把思维想像成一栋有很多房间的房子
大多数房间我们都很熟悉
但是有时候突然有一个门出现
不知从哪儿冒出来的
打开门在你面前的是一排楼梯
我们走上楼梯
并且经历一种意识被改变的过程
1902年
伟大的美国心理学家威廉·詹姆斯
写了很多各种不同的宗教信仰里的实践体验
他搜集了各种案例
并引用了不同人的言语
那些有不同信仰经历的人的言语
其中最令我兴奋的是
有一个名叫史蒂芬·柏圣文的年轻男子
他说他在1820年邂逅了耶稣
以下是柏圣文所说的话
（音乐响起）
（视频）史蒂芬·柏圣文说：我想我看到了救世主， 以人的形式展现出来
大概在我房间呆了有一秒左右
张开双臂
似乎对我说："来吧"
第二天我欢呼雀跃
我感到无比幸福以至于我说我愿意去死
这个世界没有我感情可寄托的地方
在这段时间之前
我非常自私并且自以为是
但是我现在渴望整个人类的幸福安康
而且希望怀着一颗赤忱的心
去原谅我最讨厌的敌人
乔纳森·海特：所以我们不难发现
柏圣文自己的道德观念
是如何在爬上这个楼梯的过程中而消失的
在这个较高的精神层次中
他变得怀有仁爱和宽恕之心
世界上的很多宗教信仰都找到了很多种
帮助人们爬上楼梯的方法
有些人用沉思冥想来忘掉自己
有些人用迷幻药
这个是来自于16世纪的阿兹台克滚动
这里正描述了一个男子在吃裸盖草菇（一种能引起幻觉的蘑菇）
与此同时， 他被上帝猛拉着上楼
其他人用跳舞，旋转的方式
来提升自我超越的能力
但是你并不一定要依靠某个宗教信仰来帮你爬上楼梯
很多人在大自然中找到了自我超越的方法
其他人在欢快热舞中找寻自我超越
但是有一个最为奇怪的地方就是
在战争中
很多有关战争的书籍都提到过同样一件事情
那就是没有任何东西可以像战争一样
把人们紧密联系在一起。
而这把人们聚集到一起的力量也开启了一种
一种非比寻常的自我超越的经历的可能性
我给你们放一段视频
来自格伦·格雷的一本书
格雷是在二战中服役于美国军队的一名士兵
在战争后，他采访了很多其他士兵
并且写了一本书关于士兵们的所见所闻
视频中有一段重要的段落
就是描述这样的楼梯
（视频）格伦·格雷：很多退伍军人会承认
这种在战役中共同奋斗的经历
已经成为他们生活中的重要一部分
“我”不知不觉地变成了“我们”
”我的"变成了"我们的"
并且个人信仰
也失去了它的核心重要性
我相信这完全不亚于
对永生的信心
而这种信心使自我牺牲在这些场合

我将给各位放一段
三种复苏植物复苏过程
的视频。
在视频下方
有一个时间轴 各位可以看到一切发生得多么迅速。
（掌声）
很神奇 是吧？
所以我用了21年时间 研究它们是如何做到的
这些植物如何做到干而不死？
因为很多原因我研究了 图中不同的复苏植物
在干燥和有水环境下
的状态
其中一个原因是 每一种复苏植物
比如左上角这种草
叫做画眉虫草
它是苔麸的近亲
也就是很多人熟知的 埃塞俄比亚画眉草
那是埃塞俄比亚的主要作物
它不含谷蛋白
我们想开发耐旱版本的 埃塞俄比亚画眉草。
另一个我们研究其它各种各样的植物
的内在机制是相同的么？
有一个全面的了解
系统生物学方法就是
意思是我们观察基因开关
大部分基因会制造蛋白质 所以我们研究蛋白质组。
干旱来临时植物会制造什么蛋白质？
一些蛋白质会制造 让植物新陈代谢的酶
所以我们研究代谢组。
这很重要 因为植物都是固定在土地之上的
它们利用所谓的高度协调的化工厂
保护它们不受外界环境的压力。
所以研究这些
因为干燥引起的 化学变化也非常重要。
最后我们在分子层面的研究中
我们研究了脂质体
脂质是如何变化的以应对干旱的。
这也很重要
因为所有的生物膜都是脂质的。
因为在水中所以它们保持膜状
脱离水后这些膜就会破碎。
脂质同样是开启基因的信号
我们运用生理和生化研究方法
去试验和了解我们已经在
其他研究中发现的假定保护机制。
通过这些所有的研究来尝试理解
植物如何适应它周围的自然环境。
我的科学哲学是 我需要对耐旱性的机制
有全面的理解才可以给出
对于生物应用的有意义的建议。
我确信有一些人在想
这个问题的答案是：
所有我们今天食用的作物 小麦，水稻和玉米
与祖先植株相比都是高度转基因的，
我们不认为它们是转基因作物
是因为它们一直 是用传统方式培育的。
如果你问我是不是打算把 复苏植物的基因植入作物中
时间紧迫 我们已经尝试了这些手段
准确地说 我的一些在UCT的同事
珍妮弗·汤姆森，萨尔·拉夫德恩
已经先行进行了实验
一会我将展示部分资料。
但是我们将要开展的 是一项极具野心的工作
我们的目标是启动 已经存在于每棵植株中的
整套基因
它们只是还没有 在极端干旱的环境下被激活
这种方式是否属于转基因。
我将展示第一阶段实验的部分资料
在展示之前
我需要解释一下 基因工作的原理。
也许大家都知道
基因是DNA的双链结构。
它通过紧密

AttributeError: 'float' object has no attribute 'decode'

In [61]:
df_corpus.head()

D:\Python37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [34]:
df_corpus2.head(30)

,ru,zh
0,Переводчик : Olga Volftsun Редактор : Mihail S...,翻译人员: Angelia King 校对人员: Jenny Yang
1,Музыка,（音乐）
2,Текст : битджаз,文本：节拍爵士
3,Битджаз это,节拍爵士是
4,1 . Живой лупинг,1. 现场循环
5,2 . Джазовая импровизация,2.爵士即兴表演
6,3 . `` Ручное '' управление звуком .,3.“手势表达”的声效设计
7,Акселерометры,感应器
8,на обеих руках,在每只手上的感应器
9,считывают положение руки .,读懂手语姿势


In [57]:
type(2.2) == float

True

In [58]:
df_corpus.zh[type(jieba.strdecode(df_corpus.zh)) == float]

AttributeError: 'Series' object has no attribute 'decode'